### install

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
!gdown 1wesSqUWp55XF1t-fiFIEGzOK7fyPZg_e -O train.tsv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 6.5 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=1wesSqUWp55XF1t-fiFIEGzOK7fyPZg_e
To: /content/train.tsv
100% 712k/712k [00:00<00:00, 169MB/s]


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

import os
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### load and preprocess data

In [3]:
# load
# navec_news = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
data = pd.read_csv('train.tsv', sep='\t')  
data.columns = ['title', 'label']

# split
train_data, test_data = train_test_split(data, shuffle = True, test_size=0.15, random_state=44)

# huggingface type
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# pretrained tokenizer from huggingface hub
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")

# tokenize
train_dataset = train_dataset.map(lambda examples: tokenizer(examples["title"], padding="max_length"), batched=True)
test_dataset = test_dataset.map(lambda examples: tokenizer(examples["title"], padding="max_length"), batched=True)

# to torch
train_dataset.set_format('torch')
test_dataset.set_format('torch')

  0%|          | 0/5 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


  0%|          | 0/1 [00:00<?, ?ba/s]

### model init

In [8]:
# bert = AutoModelForSequenceClassification.from_pretrained('DeepPavlov/distilrubert-tiny-cased-conversational')

bert = AutoModelForSequenceClassification.from_pretrained("Tatyana/rubert-base-cased-sentiment-new", 
                                                          problem_type = "single_label_classification",
                                                        #   attention_probs_dropout_prob = 0.1,
                                                        #   classifier_dropout = 0.1,
                                                        #   hidden_dropout_prob = 0.1,
                                                          )
tokenizer = AutoTokenizer.from_pretrained("Tatyana/rubert-base-cased-sentiment-new")

bert.classifier = nn.Linear(in_features=768, out_features=2, bias=True)
bert.num_labels = 2

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Tatyana--rubert-base-cased-sentiment-new/snapshots/4108129e25d57c9c47df9042ea74e90b79d92fc7/config.json
Model config BertConfig {
  "_name_or_path": "Tatyana/rubert-base-cased-sentiment-new",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEUTRAL",
    "1": "POSITIVE",
    "2": "NEGATIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEGATIVE": 2,
    "NEUTRAL": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attentio

### train

In [9]:
batch_size = 32
f1_score = evaluate.load('f1')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_score.compute(predictions=predictions, references=labels)

args = TrainingArguments(
    output_dir = f"bert-finetuned-fake-news",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    lr_scheduler_type = 'constant_with_warmup',
    warmup_ratio = 0.3,
    weight_decay = 0.05,
)

trainer = Trainer(
    bert,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [10]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: title, __index_level_0__. If title, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4894
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1530
  Number of trainable parameters = 177854978
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__ca

Epoch,Training Loss,Validation Loss,F1
1,No log,0.684808,0.608602
2,No log,0.656750,0.587013
3,No log,0.486575,0.755263
4,0.644600,0.403972,0.815629
5,0.644600,0.425025,0.846667
6,0.644600,0.505415,0.832313
7,0.250600,0.495243,0.868906
8,0.250600,0.597064,0.840295
9,0.250600,0.563764,0.869464
10,0.102600,0.587052,0.863109


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: title, __index_level_0__. If title, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 864
  Batch size = 32
Saving model checkpoint to bert-finetuned-fake-news/checkpoint-153
Configuration saved in bert-finetuned-fake-news/checkpoint-153/config.json
Model weights saved in bert-finetuned-fake-news/checkpoint-153/pytorch_model.bin
tokenizer config file saved in bert-finetuned-fake-news/checkpoint-153/tokenizer_config.json
Special tokens file saved in bert-finetuned-fake-news/checkpoint-153/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: title, __index_level_0__. If title, __index_level_0__ are not expected by `Be

TrainOutput(global_step=1530, training_loss=0.32777180874269773, metrics={'train_runtime': 417.0942, 'train_samples_per_second': 117.336, 'train_steps_per_second': 3.668, 'total_flos': 641614808491440.0, 'train_loss': 0.32777180874269773, 'epoch': 10.0})

In [7]:
trainer.save_model()

Saving model checkpoint to bert-finetuned-fake-news
Configuration saved in bert-finetuned-fake-news/config.json
Model weights saved in bert-finetuned-fake-news/pytorch_model.bin
tokenizer config file saved in bert-finetuned-fake-news/tokenizer_config.json
Special tokens file saved in bert-finetuned-fake-news/special_tokens_map.json


### test

In [12]:
bert = AutoModelForSequenceClassification.from_pretrained('/content/bert-finetuned-fake-news/checkpoint-1377', 
                                                          num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('/content/bert-finetuned-fake-news/checkpoint-1377')

batch_size = 32
f1_score = evaluate.load('f1')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1_score.compute(predictions=predictions, references=labels)

args = TrainingArguments(
    output_dir = f"bert-finetuned-fake-news",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.05,
)

trainer = Trainer(
    bert,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

loading configuration file /content/bert-finetuned-fake-news/checkpoint-1377/config.json
Model config BertConfig {
  "_name_or_path": "/content/bert-finetuned-fake-news/checkpoint-1377",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_ver

In [13]:
trainer.evaluate(test_dataset)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: title, __index_level_0__. If title, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 864
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.5637636780738831,
 'eval_f1': 0.8694638694638694,
 'eval_runtime': 1.4083,
 'eval_samples_per_second': 613.489,
 'eval_steps_per_second': 19.172}